In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(1234)

In [ ]:
def exponential_rng(lam=1.0):  
    """ Generates exponential random number.
    
    Keywords:
        lam (float): the rate parameter, the inverse expectation of the distribution.
    
    Returns:
        exponential random number with rate lam.
    """
    return -np.log(np.random.rand()) / lam

In [ ]:
def homogeneous_poisson_process(lam, T):
    arrivals = []
    t = exponential_rng(lam)
    while t <= T:
        arrivals.append(t)
        t += exponential_rng(lam)
    return arrivals

In [ ]:
lam_business = lambda t: 1.2*np.sin(t*np.pi/180.0)
lam_max_business = 1.2
lam_leisure = lambda t: 0.6*t/179
lam_leisure_max = 0.6
lam_economy = lambda t: 0.8*(1 + np.sin(t*np.pi/180.0 + np.pi))
lam_max_economy = 0.8

def non_homogeneous_poisson_process(lam_t, lam_max, T):
    arrivals = []
    t = exponential_rng(lam_max)
    while t <= T:
        if np.random.rand() < lam_t(t)/lam_max:
            arrivals.append(t)
        t += exponential_rng(lam_max)
    return arrivals

In [ ]:
class Passenger:
    """ Generic event.
    
    Attributes:
        time (float): Event time.
        preferences (array): fare product preference weights.
    """
    
    def __init__(self, time):
        self.time = time
        
class Business(Passenger):

    def __init__(self, time):
        super().__init__(time)
        self.preferences = [11, 15, 18, 20, 19, 15, 12, 11, 13, 7]
        
class Leisure(Passenger):

    def __init__(self, time):
        super().__init__(time)
        self.preferences = [8, 9, 11, 12, 14, 15, 16, 18, 20, 7]
        
class Economy(Passenger):

    def __init__(self, time):
        super().__init__(time)
        self.preferences = [1, 5, 8, 10, 11, 12, 13, 15, 20, 7]

In [ ]:
class Scenario:
    """ Road scenario
    
    Attributes:
        planning_horizon (int): days.
        t (int): early sales discount
        lam_business (float):
        lam_leisure (float):
        lam_economy (float):
    """
    
    def __init__(self, 
                 planning_horizon=180,
                 t=21,
                 lam_business=1.2,
                 lam_leisure=0.6,
                 lam_economy=0.8,
                 flag=0 # 1 if we want to offer 1 product at a time 
                ):
        self.planning_horizon = planning_horizon
        self.t = t
        self.lam_business = lam_business
        self.lam_leisure = lam_leisure
        self.lam_economy = lam_economy
        self.flag = flag

In [ ]:
def simulate(scenario):
    """ Implements the simulation procedure.
    
    Keywords:
        scenario (Scenario): Road scenario.
    
    Returns:
        times (list): Event times. 
        queues (list): Queue length over time. 
    """
    
    #Initialise the simulation
    t = 179
    events = []
    if scenario.flag:
        availabilities = np.zeros([1, 10])
        availabilities[-1] = 20 # Start from fare I and offer one at a time up to fare A
    else:
        availabilities = 20*np.ones([1, 10])
    revenues = 0
    
    #Trigger the first events
    print('Start of simulation')
    
    # Generate customer arrivals
    business_arrival_process = non_homogeneous_poisson_process(lam_business, lam_max_business, t)
    for i in range(business_arrival_process):
        events.append(Business(business_arrival_process[i]))
    
    leisure_arrival_process = non_homogeneous_poisson_process(lam_leisure, lam_max_leisure, t)
    for i in range(v_arrival_process):
        events.append(Leisure(leisure_arrival_process[i]))
    
    economy_arrival_process = non_homogeneous_poisson_process(lam_economy, lam_max_economy, t)
    for i in range(economy_arrival_process):
        events.append(Economy(economy_arrival_process[i]))
        
    events.sort(key=lambda event: event.time)
    
    '''
    termination = Termination(scenario.demand_duration)
    events = [generation, termination]
    '''
    
    #Main loop
    while len(events):
        e = events[0]
        
        if isinstance(e, Business):
            print('Business')
            p = e.preferences*np.where(availabilities > 0)
            p = p/np.sum(p)
                
        elif isinstance(e, Leisure):
            print('Leisure')
            
                
        elif isinstance(e, Economy):
            print('Economy')
            
        else:
            raise('Unknown event type!')
            
        events.pop(0)
        # events.sort(key=lambda event: event.time)
    
    print('End of simulation')
    
    return times, queues